In [8]:
import numpy as np              # Arrays
import random                   # Random number generation 
import matplotlib.pyplot as plt # General visualization
import networkx as nx           # Graph visualization  

import itertools

We model the floor plans of warehouses as digraphs. Nodes represent locations where stock is shelved and edges indicate an uninterrupted path between two locations. These graphs are be sparse as only nearby locations are accessible and so, given this, graphs are implemented as adjacency lists rather than matrices.

Generating an n by m floor plan with k open corridors.

In [10]:
num_rows = 10
num_cols = 5

graph = nx.Graph()
grid = list(itertools.product(list(range(num_rows)), list(range(num_cols))))
for loc in grid:
    if (loc[0] != 0):
        graph.add_edge()





[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4)]
